# 0. Import Library

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme()

# 1. Data Preprocessing
## 1.0. Data Loading

In [ ]:
df = pd.read_csv('data/kill_match_stats.csv')

In [ ]:
df.head()

In [ ]:
df.drop(columns = ['killer_name', 'victim_name', 'killer_placement', 'victim_placement', 'match_id'], inplace = True)

In [ ]:
df = df[df['map'] == 'ERANGEL'].reset_index(drop = True)

In [ ]:
df.drop(columns = ['map'], inplace = True)

In [ ]:
df.rename(columns = {
    'killer_position_x': 'kx', 
    'killer_position_y': 'ky',
    'victim_position_x': 'vx',
    'victim_position_y': 'vy'
},inplace = True)

In [ ]:
df.head()

<span style="color:red">Câu lệnh bên dưới chỉ thực hiện một lần duy nhất trên máy một thành viên duy nhất. Sau khi thực hiện, tiến hành lưu trữ df thành file kill_match_stats_v1.csv, các thành viên khác chỉ cần load file này vào là được, không cần chạy lại câu lệnh `to_csv` để tránh mất thời gian<span>

In [ ]:
df.to_csv('data/kill_match_stats_v1.csv', index = False)

## 1.1. Data Cleaning

In [ ]:
df = pd.read_csv('data/kill_match_stats_v1.csv')
df.head()

### 1.1.1. Missing values

In [ ]:
df.isnull().sum()

In [ ]:
print(f'Số dòng của df: {df.shape[0]:,}')

In [ ]:
df[df['kx'].isnull()]['killed_by'].value_counts()

In [ ]:
df.loc[df['kx'].isnull(), 'kx'] = df['vx']
df.loc[df['ky'].isnull(), 'ky'] = df['vy']

### 1.1.2. Duplicated values

In [ ]:
df.duplicated().sum()

In [ ]:
data = df[df.duplicated()]

In [ ]:
data[['kx', 'ky', 'vx', 'vy']].value_counts()

In [ ]:
df.drop_duplicates(inplace = True, ignore_index = True)

### 1.1.3. Outliers

In [ ]:
print(df['killed_by'].value_counts().sort_values())

In [ ]:
df = df[(df['killed_by'] != 'death.PlayerMale_A_C') & (df['killed_by'] != 'death.Buff_FireDOT_C') ]

In [ ]:
sns.boxplot(x = 'variable', y = 'value', data = pd.melt(df[['kx', 'ky', 'vx', 'vy']]));

In [ ]:
df = df[(df['kx'] <= 800000) & (df['kx'] >= 0) 
        & (df['ky'] <= 800000) & (df['ky'] >= 0)
        & (df['vx'] <= 800000) & (df['vx'] >= 0)
        & (df['vy'] <= 800000) & (df['vy'] >= 0)]

In [ ]:
sns.boxplot(x = df['time']);

## 1.2. Data Quality Assesement

### 1.2.1 Killed_by

In [ ]:
df.replace({
    'death.RedZoneBomb_C': 'RedZone',
    'death.ProjMolotov_C': 'Molotov',
    'death.ProjMolotov_DamageField_C': 'Molotov',
    'Bluezone': 'BlueZone'
}, inplace = True)

### 1.2.2. Các cột tọa độ

### 1.2.3. Thêm một số cột khác
#### 1.2.3.1. Distance

In [ ]:
df['dis'] = np.sqrt((df['kx'] - df['vx'])**2 + (df['ky'] - df['vy'])**2).round(0).astype(int)

In [ ]:
phase_df = pd.DataFrame({
    'time': [1, 721, 1061, 1301, 1481, 1651, 1761, 1881, 1971],
    'phase': [1, 2, 3, 4, 5, 6, 7, 8, 9],
})

In [ ]:
df['phase'] = pd.merge_asof(df[['time']].sort_values(by = ['time']).reset_index(), phase_df, on = 'time').sort_values(by = ['index']).reset_index(drop = True)['phase']

In [ ]:
type_df = pd.DataFrame(list({
    'M416': 'AR', 'SCAR-L': 'AR', 'AKM': 'AR', 'Groza': 'AR', 'M16A4': 'AR', 'AUG': 'AR',
    'SKS': 'DMR', 'Mini 14': 'DMR', 'Mk14': 'DMR', 'VSS': 'DMR',
    'Kar98k': 'SR', 'AWM': 'SR', 'M24': 'SR',
    'M249': 'LMG', 'DP-28': 'LMG',
    'UMP9': 'SMG', 'Vector': 'SMG', 'Tommy Gun': 'SMG', 'Micro UZI': 'SMG',
    'S1897': 'Shotgun', 'S686': 'Shotgun', 'S12K': 'Shotgun',
    'P1911': 'Handgun', 'R1895': 'Handgun', 'P92': 'Handgun', 'P18C': 'Handgun', 'Crossbow': 'Handgun',
    'Pan': 'Melee', 'Machete': 'Melee', 'Crowbar': 'Melee', 'Sickle': 'Melee', 'Punch': 'Melee',
    'BlueZone': 'Zone', 'RedZone': 'Zone',
    'Molotov': 'Throwable', 'Grenade': 'Throwable',
    'Motorbike': 'Vehicle', 'Motorbike (SideCar)': 'Vehicle', 'Dacia': 'Vehicle', 'Uaz': 'Vehicle',
    'Buggy': 'Vehicle', 'Boat': 'Vehicle', 'Aquarail': 'Vehicle',
    'Falling': 'Self', 'Drown': 'Self'
}.items()), columns = ['killed_by', 'type'])

In [ ]:
df = df.join(type_df.set_index('killed_by'), on = 'killed_by', how = 'left')
df = df[(df['killed_by'] != 'Hit by Car') & (df['killed_by'] != 'Down and Out')]

#### 1.2.3.4. More of Data Cleaning
##### 1.2.3.4.1. Outlier Distance

In [ ]:
plt.figure(figsize = (15, 5))
sns.boxplot(data = df, x = 'type', y = 'dis')

In [ ]:
plt.figure(figsize = (15, 5))

type_index = df[(
    ((df['type'] == 'Self') & (df['dis'] > 20)) |
    ((df['type'] == 'AR') & (df['dis'] > 500)) |
    ((df['type'] == 'DMR') & (df['dis'] > 600)) |
    ((df['type'] == 'SR') & (df['dis'] > 1000)) |
    ((df['type'] == 'Shotgun') & (df['dis'] > 100)) |
    ((df['type'] == 'Handgun') & (df['dis'] > 75)) |
    ((df['type'] == 'SMG') & (df['dis'] > 200)) |
    ((df['type'] == 'LMG') & (df['dis'] > 400)) |
    ((df['type'] == 'Throwable') & (df['dis'] > 100)) |
    ((df['type'] == 'Zone') & (df['dis'] > 10)) |
    ((df['type'] == 'Melee') & (df['dis'] > 10)) |
    ((df['type'] == 'Vehicle') & (df['dis'] > 100))
)].index

df = df.drop(type_index).reset_index(drop = True)
sns.boxplot(data = df, x = 'type', y = 'dis')


##### 1.2.3.4.2. Missing và Outlier Phase

In [ ]:
df = df[df['time'] <= 2160]

In [ ]:
df['phase'].isnull().sum()

In [ ]:
df = df[df['phase'].notnull()]
df['phase'] = df['phase'].astype(int)
df.reset_index(drop = True, inplace = True)

In [ ]:
df = df.iloc[:, [0, 8, 3, 7, 6, 1, 2, 4, 5]]

df.to_csv('data/kill_match_stats_v2.csv', index = False)